# Module 12 Challenge
## Deliverable 2: Scrape and Analyze Mars Weather Data

In [1]:
# Import relevant libraries
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

DriverNotFoundError: Driver for chrome was not found.

### Step 1: Visit the Website

Use automated browsing to visit the [Mars Temperature Data Site](https://data-class-mars-challenge.s3.amazonaws.com/Mars/index.html). Inspect the page to identify which elements to scrape. Note that the URL is `https://data-class-mars-challenge.s3.amazonaws.com/Mars/index.html`.

   > **Hint** To identify which elements to scrape, you might want to inspect the page by using Chrome DevTools to discover whether the table contains usable classes.


In [ ]:
# Visit the website
# https://data-class-mars-challenge.s3.amazonaws.com/Mars/index.html
url = 'https://data-class-mars-challenge.s3.amazonaws.com/Mars/index.html'
browser.visit(url)



### Step 2: Scrape the Table

Create a Beautiful Soup object and use it to scrape the data in the HTML table.

Note that this can also be achieved by using the Pandas `read_html` function. However, use Beautiful Soup here to continue sharpening your web scraping skills.

In [ ]:
# Create a Beautiful Soup Object
html=browser.html
htmlsoup=soup(html, 'html.parser')

table = soup.find('table', class_='table')
table

In [ ]:
# Extract all rows of data
rows = htmlsoup.find_all('tr', class_='data-row')

##dicts = {}
#rows = table.find_all('tr')

##keys = row.find('th')
##values = row('td')

##for i in keys:
   ## for x in values:
      ##  dicts[i]=x

#for row in rows:
 #   row_heading = row.find('th')#.text
  #  row_data = row.find('td')#.text.strip()
   # mars_facts[row_heading] = row_data

In [ ]:
print(rows)

### Step 3: Store the Data

Assemble the scraped data into a Pandas DataFrame. The columns should have the same headings as the table on the website. Here’s an explanation of the column headings:

* `id`: the identification number of a single transmission from the Curiosity rover
* `terrestrial_date`: the date on Earth
* `sol`: the number of elapsed sols (Martian days) since Curiosity landed on Mars
* `ls`: the solar longitude
* `month`: the Martian month
* `min_temp`: the minimum temperature, in Celsius, of a single Martian day (sol)
* `pressure`: The atmospheric pressure at Curiosity's location

In [ ]:
# Create an empty list
#data = [[i for i in mars_facts[x]] for x in mars_facts.keys()]

data = []

# Loop through the scraped data to create a list of rows
for row in rows:
    facts = row.find_all('td')
    row = [col.text for col in facts]
    data.append(row)
    
print(data)

In [ ]:
# Create a Pandas DataFrame by using the list of rows and a list of the column names
mars_headers = []
for header in headers:
    column_names = header.find('th')
    mars_headers.append(header.text.replace("/n", " "))
mars_headers

In [ ]:
mars_df = pd.DataFrame(mars_data, columns = ['id', 'terrestrial_date', 'sol', 'ls', 'month', 'min_temp', 'pressure'])

In [ ]:
# Confirm DataFrame was created successfully
mars_df

### Step 4: Prepare Data for Analysis

Examine the data types that are currently associated with each column. If necessary, cast (or convert) the data to the appropriate `datetime`, `int`, or `float` data types.

  > **Hint** You can use the Pandas `astype` and `to_datetime` methods to accomplish this task.


In [ ]:
# Examine data type of each column
mars_df.dtypes

In [ ]:
# Change data types for data analysis
# Change data types for data analysis
mars_df[["id","sol","ls","month"]] = mars_df[["id","sol","ls","month"]].astype(int)
mars_df[["min_temp","pressure"]] = mars_df[["min_temp","pressure"]].astype(float)
mars_df["terrestrial_date"] = pd.to_datetime(mars_df["terrestrial_date"])

In [ ]:
# Confirm type changes were successful by examining data types again
mars_df.dtypes

### Step 5: Analyze the Data

Analyze your dataset by using Pandas functions to answer the following questions:

1. How many months exist on Mars?
2. How many Martian (and not Earth) days worth of data exist in the scraped dataset?
3. What are the coldest and the warmest months on Mars (at the location of Curiosity)? To answer this question:
    * Find the average the minimum daily temperature for all of the months.
    * Plot the results as a bar chart.
4. Which months have the lowest and the highest atmospheric pressure on Mars? To answer this question:
    * Find the average the daily atmospheric pressure of all the months.
    * Plot the results as a bar chart.
5. About how many terrestrial (Earth) days exist in a Martian year? To answer this question:
    * Consider how many days elapse on Earth in the time that Mars circles the Sun once.
    * Visually estimate the result by plotting the daily minimum temperature.


In [ ]:
# 1. How many months are there on Mars?
mars_df["month"].max()

In [ ]:
# 2. How many Martian days' worth of data are there?
mars_df["sol"].max()

In [ ]:
# 3. What is the average low temperature by month?
min_temp = mars_df.groupby(["month"]).mean()["min_temp"]
min_temp

In [ ]:
# Plot the average temperature by month
from matplotlib import pyplot as plt
min_temp.plot(x = "month", y = "min_temp", kind="bar")
plt.title("Average Low Temperature by Martian Month")
plt.xlabel("Month")
plt.ylabel("Temperature (°C)")
plt.legend(["Average Minimum Temperature"])
plt.show()

In [ ]:
# Identify the coldest and hottest months in Curiosity's location
min_temp_df = pd.DataFrame(min_temp, columns = ["min_temp"])
min_temp_df.reset_index(inplace=False)

coldest_month = min_temp_df["min_temp"].min()
coldest_month_row = min_temp_df.loc[min_temp_df["min_temp"] == coldest_month]

hottest_month = min_temp_df["min_temp"].max()
hottest_month_row = min_temp_df.loc[min_temp_df["min_temp"] == hottest_month]

print(f"Coldest Month:  {coldest_month_row}")
print(f"Hottest Month:  {hottest_month_row}")

In [ ]:
# 4. Average pressure by Martian month
avg_pressure = mars_table_df.groupby(["month"]).mean()["pressure"]
avg_pressure

In [ ]:
# Plot the average pressure by month
avg_pressure.plot(x = "month", y = "pressure", kind="bar")
plt.title("Average Pressure by Martian Month")
plt.xlabel("Month")
plt.ylabel("Pressure (Pa)")
plt.show()

In [ ]:
# 5. How many terrestrial (earth) days are there in a Martian year?
mars_df.plot(x = "sol", y = "min_temp")
plt.title("Daily Minimum Temperature")
plt.xlabel("Days Since Curiosity Landed")
plt.ylabel("Temperature (°C)")
plt.xticks(np.arange(0,2000, step=200))
plt.show()

On average, the third month has the coldest minimum temperature on Mars, and the eighth month is the warmest. But it is always very cold there in human terms!



Atmospheric pressure is, on average, lowest in the sixth month and highest in the ninth.

The distance from peak to peak is roughly 1425-750, or 675 days. A year on Mars appears to be about 675 days from the plot. Internet search confirms that a Mars year is equivalent to 687 earth days.

### Step 6: Save the Data

Export the DataFrame to a CSV file.

In [ ]:
# Write the data to a CSV
mars_table_df.to_csv("mars_data.csv")

In [ ]:
browser.quit()